In [27]:
# ======================
# 1. 라이브러리 및 패키지 임포트
# ======================

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import yfinance as yf
#from pandas_datareader import data as pdr
from sklearn.preprocessing import StandardScaler
from datetime import datetime

In [28]:
# ======================
# 2. 각 데이터 불러오기 (gold_data 폴더에서)
# ======================

print("gold_data 폴더에서 데이터 로딩 시작...")

# 데이터 경로
data_path = "./gold_data/"

# 각 파일별로 데이터 로딩
data_files = {}

# 01번부터 13번까지 파일 로딩
for i in range(1, 14):
    file_num = f"{i:02d}"  # 01, 02, 03, ... 13
    try:
        file_path = f"{data_path}{file_num}*.csv"
        import glob
        matching_files = glob.glob(file_path)
        
        if matching_files:
            # 첫 번째 매칭 파일 사용
            df = pd.read_csv(matching_files[0], index_col=0, parse_dates=True)
            data_files[f"data_{file_num}"] = df
            print(f"{file_num}번 파일 로딩 완료: {df.shape}")
        else:
            print(f"{file_num}번 파일을 찾을 수 없습니다.")
            
    except Exception as e:
        print(f"{file_num}번 파일 로딩 실패: {e}")

print(f"총 {len(data_files)}개 파일 로딩 완료")

# 로딩된 데이터 확인
for key, df in data_files.items():
    print(f"{key}: {df.shape}, 컬럼: {list(df.columns)}")

gold_data 폴더에서 데이터 로딩 시작...
01번 파일 로딩 완료: (186, 5)
02번 파일 로딩 완료: (186, 1)
03번 파일 로딩 완료: (186, 1)
04번 파일 로딩 완료: (186, 2)
05번 파일 로딩 완료: (186, 10)
06번 파일 로딩 완료: (186, 2)
07번 파일을 찾을 수 없습니다.
08번 파일 로딩 완료: (186, 5)
09번 파일 로딩 완료: (186, 6)
10번 파일 로딩 완료: (186, 5)
11번 파일 로딩 완료: (186, 5)
12번 파일 로딩 완료: (186, 5)
13번 파일 로딩 완료: (186, 1)
총 12개 파일 로딩 완료
data_01: (186, 5), 컬럼: ['Close', 'High', 'Low', 'Open', 'Volume']
data_02: (186, 1), 컬럼: ['M2_Money_Supply_Billions_USD']
data_03: (186, 1), 컬럼: ['CPIAUCNS']
data_04: (186, 2), 컬럼: ['Close', 'Volume']
data_05: (186, 10), 컬럼: ['Gold_Reserves_United_States', 'Gold_Reserves_Germany', 'Gold_Reserves_Italy', 'Gold_Reserves_France', 'Gold_Reserves_Russia', 'Gold_Reserves_China', 'Gold_Reserves_Switzerland', 'Gold_Reserves_Japan', 'Gold_Reserves_India', 'Gold_Reserves_Netherlands']
data_06: (186, 2), 컬럼: ['USD_CNY_Rate', 'USD_EUR_Rate']
data_08: (186, 5), 컬럼: ['Close', 'High', 'Low', 'Open', 'Volume']
data_09: (186, 6), 컬럼: ['Price', 'Close', 'High', 'Low', 'O

In [ ]:
# ======================
# 2-1. 필요한 컬럼만 추출 (각 파일별 최적화)
# ======================

print("필요한 컬럼 추출 중...")

extracted_data = {}

# 컬럼 찾기 함수
def find_column(df, target_names):
    """대소문자 구분 없이 컬럼 찾기"""
    for target in target_names:
        for col in df.columns:
            if col.upper() == target.upper():
                return col
    return None

# 각 파일별로 필요한 컬럼 추출
for i in range(1, 14):
    file_num = f"{i:02d}"
    data_key = f"data_{file_num}"
    
    if data_key in data_files:
        df = data_files[data_key]
        
        # 파일별 컬럼 추출 규칙 (실제 데이터 구조에 맞게)
        if file_num == "01":  # 금시세 (종속변수)
            target_col = find_column(df, ['Close', 'close', 'Price', 'price'])
            if target_col:
                extracted_data[f"feature_{file_num}"] = df[target_col]
                print(f"{file_num}: {target_col} 추출 (금시세)")
            else:
                first_col = df.columns[0]
                extracted_data[f"feature_{file_num}"] = df[first_col]
                print(f"{file_num}: {first_col} 추출")
                
        elif file_num == "02":  # M2통화량
            target_col = find_column(df, ['M2_Money_Supply_Billions_USD', 'M2', 'm2'])
            if target_col:
                extracted_data[f"feature_{file_num}"] = df[target_col]
                print(f"{file_num}: {target_col} 추출")
            else:
                first_col = df.columns[0]
                extracted_data[f"feature_{file_num}"] = df[first_col]
                print(f"{file_num}: {first_col} 추출")
                
        elif file_num == "03":  # 미국 CPI
            target_col = find_column(df, ['CPI', 'cpi', 'Consumer_Price_Index'])
            if target_col:
                extracted_data[f"feature_{file_num}"] = df[target_col]
                print(f"{file_num}: {target_col} 추출")
            else:
                first_col = df.columns[0]
                extracted_data[f"feature_{file_num}"] = df[first_col]
                print(f"{file_num}: {first_col} 추출")
                
        elif file_num == "04":  # 거래량
            target_col = find_column(df, ['Volume', 'VOLUME', 'volume'])
            if target_col:
                extracted_data[f"feature_{file_num}"] = df[target_col]
                print(f"{file_num}: {target_col} 추출")
            else:
                first_col = df.columns[0]
                extracted_data[f"feature_{file_num}"] = df[first_col]
                print(f"{file_num}: {first_col} 추출")
                
        elif file_num == "05":  # 금 보유량 (모든 컬럼)
            for col in df.columns:
                feature_name = f"feature_{file_num}_{col}"
                extracted_data[feature_name] = df[col]
            print(f"{file_num}: {len(df.columns)}개 컬럼 추출")
                
        elif file_num == "06":  # 환율 (모든 컬럼)
            for col in df.columns:
                feature_name = f"feature_{file_num}_{col}"
                extracted_data[feature_name] = df[col]
            print(f"{file_num}: {len(df.columns)}개 컬럼 추출")
            
        elif file_num == "07":  # 파일 없음
            continue
                
        elif file_num == "08":  # 은 가격
            target_col = find_column(df, ['Close', 'close', 'Price', 'price'])
            if target_col:
                extracted_data[f"feature_{file_num}"] = df[target_col]
                print(f"{file_num}: {target_col} 추출")
            else:
                first_col = df.columns[0]
                extracted_data[f"feature_{file_num}"] = df[first_col]
                print(f"{file_num}: {first_col} 추출")
                
        elif file_num == "09":  # 달러지수
            target_col = find_column(df, ['Close', 'close', 'Index', 'index', 'Value', 'value'])
            if target_col:
                extracted_data[f"feature_{file_num}"] = df[target_col]
                print(f"{file_num}: {target_col} 추출")
            else:
                first_col = df.columns[0]
                extracted_data[f"feature_{file_num}"] = df[first_col]
                print(f"{file_num}: {first_col} 추출")
                
        elif file_num == "10":  # S&P 500
            target_col = find_column(df, ['Close', 'close', 'Price', 'price'])
            if target_col:
                extracted_data[f"feature_{file_num}"] = df[target_col]
                print(f"{file_num}: {target_col} 추출")
            else:
                first_col = df.columns[0]
                extracted_data[f"feature_{file_num}"] = df[first_col]
                print(f"{file_num}: {first_col} 추출")
                
        elif file_num == "11":  # 나스닥
            target_col = find_column(df, ['Close', 'close', 'Price', 'price'])
            if target_col:
                extracted_data[f"feature_{file_num}"] = df[target_col]
                print(f"{file_num}: {target_col} 추출")
            else:
                first_col = df.columns[0]
                extracted_data[f"feature_{file_num}"] = df[first_col]
                print(f"{file_num}: {first_col} 추출")
                
        elif file_num == "12":  # 채권금리/가격
            target_col = find_column(df, ['close', 'Close', 'rate', 'Rate', 'yield', 'Yield'])
            if target_col:
                extracted_data[f"feature_{file_num}"] = df[target_col]
                print(f"{file_num}: {target_col} 추출")
            else:
                first_col = df.columns[0]
                extracted_data[f"feature_{file_num}"] = df[first_col]
                print(f"{file_num}: {first_col} 추출")
                
        elif file_num == "13":  # VIX 공포지수
            target_col = find_column(df, ['VIX_Fear_Index', 'VIX', 'vix', 'Fear_Index'])
            if target_col:
                extracted_data[f"feature_{file_num}"] = df[target_col]
                print(f"{file_num}: {target_col} 추출")
            else:
                first_col = df.columns[0]
                extracted_data[f"feature_{file_num}"] = df[first_col]
                print(f"{file_num}: {first_col} 추출")

print(f"추출된 피쳐 수: {len(extracted_data)}개")

필요한 컬럼 추출 중...

01번 파일 컬럼: ['Close', 'High', 'Low', 'Open', 'Volume']
01번: Close 추출 (종속변수)

02번 파일 컬럼: ['M2_Money_Supply_Billions_USD']
02번: M2_Money_Supply_Billions_USD 추출 (CLOSE 없어서 첫번째 컬럼 사용)

03번 파일 컬럼: ['CPIAUCNS']
03번: CPIAUCNS 추출 (CLOSE 없어서 첫번째 컬럼 사용)

04번 파일 컬럼: ['Close', 'Volume']
04번: Volume 추출

05번 파일 컬럼: ['Gold_Reserves_United_States', 'Gold_Reserves_Germany', 'Gold_Reserves_Italy', 'Gold_Reserves_France', 'Gold_Reserves_Russia', 'Gold_Reserves_China', 'Gold_Reserves_Switzerland', 'Gold_Reserves_Japan', 'Gold_Reserves_India', 'Gold_Reserves_Netherlands']
05번: 모든 컬럼 추출 (10개)

06번 파일 컬럼: ['USD_CNY_Rate', 'USD_EUR_Rate']
06번: 모든 컬럼 추출 (2개)

08번 파일 컬럼: ['Close', 'High', 'Low', 'Open', 'Volume']
08번: Close 추출

09번 파일 컬럼: ['Price', 'Close', 'High', 'Low', 'Open', 'Volume']
09번: Close 추출

10번 파일 컬럼: ['Close', 'High', 'Low', 'Open', 'Volume']
10번: Close 추출

11번 파일 컬럼: ['Open', 'High', 'Low', 'Close', 'Volume']
11번: Close 추출

12번 파일 컬럼: ['close', 'open', 'high', 'low', 'change_pct']

In [ ]:
# ======================
# 2-2. 모든 데이터 날짜 형식 통일 (YYYY-MM)
# ======================

print("데이터 날짜 형식 통일 중...")

# 날짜 인덱스를 YYYY-MM 문자열로 변환하는 함수
def convert_to_yyyymm_string(series, series_name):
    """모든 형태의 날짜 인덱스를 YYYY-MM 문자열로 변환"""
    try:
        # 이미 문자열이고 YYYY-MM 형식인지 확인
        if isinstance(series.index[0], str) and len(series.index[0]) == 7 and '-' in series.index[0]:
            return series
        
        # DatetimeIndex가 아닌 경우 변환
        if not isinstance(series.index, pd.DatetimeIndex):
            series.index = pd.to_datetime(series.index, utc=True)
        
        # timezone 제거
        if series.index.tz is not None:
            series.index = series.index.tz_convert(None)
        
        # Period로 변환하여 월 단위로 집계
        period_index = series.index.to_period('M')
        
        # 같은 월에 여러 데이터가 있으면 평균값 사용
        if len(period_index.unique()) < len(period_index):
            temp_df = pd.DataFrame({'value': series.values, 'period': period_index})
            monthly_data = temp_df.groupby('period')['value'].mean()
        else:
            monthly_data = pd.Series(series.values, index=period_index)
        
        # Period를 YYYY-MM 문자열로 변환
        monthly_data.index = monthly_data.index.astype(str)
        monthly_data.name = series_name
        
        return monthly_data
        
    except Exception as e:
        print(f"{series_name} 변환 실패: {e}")
        return series

# 모든 extracted_data를 YYYY-MM 문자열 형식으로 변환
standardized_data = {}

for key, series in extracted_data.items():
    standardized_series = convert_to_yyyymm_string(series.copy(), key)
    standardized_data[key] = standardized_series

# extracted_data 업데이트
extracted_data = standardized_data

print("데이터 변환 완료")

# 공통 날짜 범위 계산
try:
    valid_series = []
    for series in extracted_data.values():
        if len(series) > 0:
            valid_series.append(series)
    
    if valid_series:
        # 모든 시작/끝 날짜 수집
        all_start_dates = [series.index[0] for series in valid_series]
        all_end_dates = [series.index[-1] for series in valid_series]
        
        # 공통 범위 찾기
        common_start = max(all_start_dates)
        common_end = min(all_end_dates)
        
        print(f"공통 기간: {common_start} ~ {common_end}")
        
        # 공통 기간 내 월 수 계산
        start_year, start_month = map(int, common_start.split('-'))
        end_year, end_month = map(int, common_end.split('-'))
        months_count = (end_year - start_year) * 12 + (end_month - start_month) + 1
        print(f"공통 기간 월 수: {months_count}개월")
            
except Exception as e:
    print(f"공통 날짜 범위 계산 실패: {e}")

모든 데이터 날짜 형식 통일 중... (YYYY-MM 문자열 형태)

총 22개 데이터 처리 시작:

feature_01 처리 중...
  feature_01 변환 시작...
    원본 인덱스 타입: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
    월별 집계 중...
    변환 완료: 186개, 2010-01 ~ 2025-06

feature_02 처리 중...
  feature_02 변환 시작...
    원본 인덱스 타입: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
    월별 집계 중...
    변환 완료: 186개, 2010-01 ~ 2025-06

feature_03 처리 중...
  feature_03 변환 시작...
    원본 인덱스 타입: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
    월별 집계 중...
    변환 완료: 186개, 2010-01 ~ 2025-06

feature_04 처리 중...
  feature_04 변환 시작...
    원본 인덱스 타입: <class 'pandas.core.indexes.base.Index'>
    DatetimeIndex로 변환 중...
    timezone 제거 중...
    월별 집계 중...
    변환 완료: 186개, 2010-01 ~ 2025-06

feature_05_Gold_Reserves_United_States 처리 중...
  feature_05_Gold_Reserves_United_States 변환 시작...
    원본 인덱스 타입: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
    월별 집계 중...
    변환 완료: 186개, 2010-01 ~ 2025-06

feature_05_Gold_Reserves_Germany 처리 중...
  feat

In [ ]:
# ======================
# 3. 데이터 통합 및 병합
# ======================

print("데이터 통합 중...")

# 추출된 데이터들을 DataFrame으로 병합
all_features = []
feature_names = []

# 피쳐 이름 매핑
feature_name_mapping = {
    'feature_01': 'Gold_Price',              # 금시세 (종속변수)
    'feature_02': 'US_M2_Money_Supply',      # 미국 M2통화량
    'feature_03': 'US_CPI',                  # 미국 소비자물가지수
    'feature_04': 'Gold_Volume',             # 금 거래량
    'feature_05': 'Gold_Reserves',           # 금 보유량 (다중 컬럼)
    'feature_06': 'Exchange_Rates',          # 환율 (다중 컬럼)
    'feature_08': 'Silver_Price',            # 은 가격
    'feature_09': 'Dollar_Index',            # 달러지수
    'feature_10': 'SP500',                   # S&P 500 지수
    'feature_11': 'NASDAQ',                  # 나스닥 지수
    'feature_12': 'Bond_Rate',               # 채권금리/가격
    'feature_13': 'VIX_Fear_Index'           # VIX 공포지수
}

# 피쳐 정렬 및 이름 생성
sorted_keys = sorted(extracted_data.keys(), key=lambda x: (
    int(x.split('_')[1]), x.split('_')[2] if len(x.split('_')) > 2 else ''
))

for feature_key in sorted_keys:
    series = extracted_data[feature_key]
    
    # 빈 시리즈는 건너뛰기
    if len(series) == 0:
        continue
    
    # 의미있는 피쳐 이름 생성
    if feature_key.startswith('feature_05_'):
        # 금 보유량
        suffix = feature_key.replace('feature_05_', '')
        clean_suffix = suffix.replace('Gold_Reserves_', '').replace('_', ' ')
        meaningful_name = f'Gold_Reserves_{clean_suffix}'
        
    elif feature_key.startswith('feature_06_'):
        # 환율
        suffix = feature_key.replace('feature_06_', '')
        clean_suffix = suffix.replace('USD_', '').replace('_Rate', '').replace('_', '/')
        meaningful_name = f'Exchange_Rate_USD_{clean_suffix}'
        
    else:
        # 단일 컬럼 파일들
        base_key = '_'.join(feature_key.split('_')[:2])
        meaningful_name = feature_name_mapping.get(base_key, feature_key)
    
    series.name = meaningful_name
    all_features.append(series)
    feature_names.append(meaningful_name)

# 피쳐 병합
if all_features:
    print(f"{len(all_features)}개 피쳐 병합 시작...")
    
    # 공통 날짜 범위 찾기
    start_dates = [feature.index[0] for feature in all_features if len(feature) > 0]
    end_dates = [feature.index[-1] for feature in all_features if len(feature) > 0]
    
    if start_dates and end_dates:
        common_start = max(start_dates)
        common_end = min(end_dates)
        
        print(f"공통 기간: {common_start} ~ {common_end}")
        
        # 공통 기간으로 필터링
        filtered_features = []
        for feature in all_features:
            mask = (feature.index >= common_start) & (feature.index <= common_end)
            filtered_feature = feature[mask]
            
            if len(filtered_feature) > 0:
                filtered_features.append(filtered_feature)
        
        # 피쳐 병합
        if filtered_features:
            combined_data = pd.concat(filtered_features, axis=1, join='outer')
            
            # VIX 중복 컬럼 처리
            if 'Bond_Rate' in combined_data.columns and 'VIX_Fear_Index' in combined_data.columns:
                if combined_data['Bond_Rate'].equals(combined_data['VIX_Fear_Index']):
                    print("Bond_Rate와 VIX_Fear_Index 동일 - Bond_Rate 제거")
                    combined_data = combined_data.drop(columns=['Bond_Rate'])
            
            # 결측치 확인
            missing_counts = combined_data.isnull().sum()
            total_missing = missing_counts.sum()
            
            if total_missing > 0:
                print(f"총 결측치: {total_missing}개")
                for col, missing in missing_counts.items():
                    if missing > 0:
                        missing_pct = (missing / len(combined_data)) * 100
                        print(f"  {col}: {missing}개 ({missing_pct:.1f}%)")
            
            print(f"최종 데이터 크기: {combined_data.shape}")
            print(f"기간: {combined_data.index[0]} ~ {combined_data.index[-1]}")
            
            # 인덱스 설정 및 저장
            combined_data.index.name = 'Date_YYYY_MM'
            raw_filename = "gold_data_raw_combined.csv"
            combined_data.to_csv(raw_filename, encoding='utf-8-sig')
            print(f"데이터 저장: {raw_filename}")
            
            # 컬럼 정보
            print(f"\n최종 피쳐 목록 ({len(combined_data.columns)}개):")
            for i, col in enumerate(combined_data.columns, 1):
                print(f"  {i:2d}. {col}")
            
        else:
            print("필터링된 데이터가 없습니다.")
            combined_data = pd.DataFrame()
    else:
        print("공통 날짜 범위를 찾을 수 없습니다.")
        combined_data = pd.DataFrame()
    
else:
    print("병합할 데이터가 없습니다.")
    combined_data = pd.DataFrame()

데이터 통합 중... (YYYY-MM 문자열 인덱스)

추출된 데이터 정보:
feature_01: 186개, 2010-01 ~ 2025-06
feature_02: 186개, 2010-01 ~ 2025-06
feature_03: 186개, 2010-01 ~ 2025-06
feature_04: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_United_States: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_Germany: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_Italy: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_France: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_Russia: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_China: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_Switzerland: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_Japan: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_India: 186개, 2010-01 ~ 2025-06
feature_05_Gold_Reserves_Netherlands: 186개, 2010-01 ~ 2025-06
feature_06_USD_CNY_Rate: 186개, 2010-01 ~ 2025-06
feature_06_USD_EUR_Rate: 186개, 2010-01 ~ 2025-06
feature_08: 186개, 2010-01 ~ 2025-06
feature_09: 186개, 2010-01 ~ 2025-06
feature_10: 186개, 2010-01 ~ 2025-06
feature_11: 186개, 2010-

PermissionError: [Errno 13] Permission denied: 'gold_data_raw_combined.csv'

In [ ]:
# ======================
# 4. 결측치 처리 및 보간
# ======================

print("결측치 처리 중...")

if not combined_data.empty:
    # 결측치 확인
    missing_info = combined_data.isnull().sum()
    total_missing = missing_info.sum()
    
    print(f"전체 데이터 크기: {combined_data.shape}")
    print(f"총 결측치: {total_missing}개")
    
    if total_missing > 0:
        print("컬럼별 결측치:")
        for col, missing_count in missing_info.items():
            if missing_count > 0:
                missing_pct = (missing_count / len(combined_data)) * 100
                print(f"  {col}: {missing_count}개 ({missing_pct:.1f}%)")
        
        # 결측치가 50% 이상인 컬럼 제거
        high_missing_cols = []
        for col, missing_count in missing_info.items():
            missing_pct = (missing_count / len(combined_data)) * 100
            if missing_pct > 50:
                high_missing_cols.append(col)
        
        if high_missing_cols:
            print(f"결측치 50% 이상 컬럼 제거: {high_missing_cols}")
            combined_data = combined_data.drop(columns=high_missing_cols)
            print(f"제거 후 데이터 크기: {combined_data.shape}")
        
        # 선형 보간법 적용
        print("선형 보간법 적용 중...")
        data_interpolated = combined_data.interpolate(method='linear', limit_direction='both')
        
        # 앞뒤 값으로 채우기
        data_filled = data_interpolated.fillna(method='ffill').fillna(method='bfill')
        
        # 최종 결측치 확인
        remaining_missing = data_filled.isnull().sum().sum()
        print(f"보간 후 남은 결측치: {remaining_missing}개")
        
        if remaining_missing > 0:
            print("남은 결측치가 있는 행 제거...")
            remaining_nulls = data_filled.isnull().sum()
            for col, null_count in remaining_nulls.items():
                if null_count > 0:
                    print(f"  {col}: {null_count}개")
            
            data_filled = data_filled.dropna()
            print(f"결측치 제거 후 데이터 크기: {data_filled.shape}")
    else:
        print("결측치 없음")
        data_filled = combined_data.copy()
    
    # 최종 데이터
    final_data = data_filled.copy()
    
    print(f"최종 데이터 크기: {final_data.shape}")
    if len(final_data) > 0:
        print(f"최종 기간: {final_data.index[0]} ~ {final_data.index[-1]}")
        
        # 기초 통계
        print("\n기초 통계 (처음 5개 컬럼):")
        print(final_data.iloc[:, :5].describe().round(2))
        
        print("\n컬럼별 데이터 요약:")
        for col in final_data.columns:
            data_count = len(final_data[col])
            data_mean = final_data[col].mean()
            data_std = final_data[col].std()
            print(f"  {col}: {data_count}개, 평균={data_mean:.2f}, 표준편차={data_std:.2f}")
    else:
        print("최종 데이터가 비어있습니다.")
    
else:
    print("처리할 데이터가 없습니다.")
    final_data = pd.DataFrame()

 결측치 처리 중...
 결측치 현황:
전체 데이터 크기: (186, 22)
총 결측치: 0개
결측치가 없습니다!

 최종 데이터 크기: (186, 22)
 최종 기간: 2010-01 ~ 2025-06

 기초 통계 (처음 5개 컬럼):
       Gold_Price  US_M2_Money_Supply  US_CPI   Gold_Volume  \
count      186.00              186.00  186.00  1.860000e+02   
mean      1593.92            14849.13  256.27  2.025577e+08   
std        440.58             4566.86   30.35  9.028885e+07   
min       1069.95             8478.00  216.69  9.735590e+07   
25%       1266.34            10999.12  233.75  1.408006e+08   
50%       1483.25            13752.65  246.59  1.789454e+08   
75%       1805.52            20672.80  273.43  2.353245e+08   
max       3353.92            22021.40  322.56  7.685474e+08   

       Gold_Reserves_Gold_Reserves_United_States  
count                                     186.00  
mean                                     8129.73  
std                                       153.46  
min                                      7707.35  
25%                                      802

In [33]:
final_data

,Gold_Price,US_M2_Money_Supply,US_CPI,Gold_Volume,Gold_Reserves_Gold_Reserves_United_States,Gold_Reserves_Gold_Reserves_Germany,Gold_Reserves_Gold_Reserves_Italy,Gold_Reserves_Gold_Reserves_France,Gold_Reserves_Gold_Reserves_Russia,Gold_Reserves_Gold_Reserves_China,...,Gold_Reserves_Gold_Reserves_India,Gold_Reserves_Gold_Reserves_Netherlands,Exchange_Rate_USD_CNY_Rate,Exchange_Rate_USD_EUR_Rate,Dollar_Index,SP500,NASDAQ,Bond_Rate,VIX,Additional_Economic
Date_YYYY_MM,,,,,,,,,,,,,,,,,,,,,
2010-01,1116.826300,8478.0,216.687,347218200,8214.300491,3392.470250,2476.156875,2461.108854,2322.747858,2257.607096,...,767.954029,618.584748,6.826916,1.426574,15.900000,79.500000,1073.869995,2147.350098,3.588,20.643158
2010-02,1098.189479,8527.6,216.741,368010500,8111.008546,3349.811128,2445.020072,2430.161274,2293.540119,2229.218480,...,758.297277,610.806262,6.828463,1.367995,16.070000,81.339996,1104.489990,2238.260010,3.619,22.540000
2010-03,1114.356525,8523.7,217.631,326092400,8238.859494,3402.613011,2483.560055,2468.467044,2329.692377,2264.356859,...,770.250047,620.434185,6.826183,1.357004,17.139999,82.239998,1169.430054,2397.959961,3.833,17.767391
2010-04,1151.452375,8555.1,218.009,274413100,8381.251267,3461.420192,2526.483292,2511.129429,2369.956327,2303.491619,...,783.562238,631.157116,6.825550,1.341682,18.260000,82.709999,1186.689941,2461.189941,3.659,17.424286
2010-05,1203.955005,8609.3,218.178,504968200,8095.410271,3343.369108,2440.318055,2425.487833,2289.129413,2224.931471,...,756.838995,609.631621,6.827450,1.256315,18.030001,87.459999,1089.410034,2257.040039,3.303,31.929500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02,2897.115800,21584.2,319.082,178992600,7994.066180,3301.514441,2409.768422,2395.123855,2260.472467,2197.078200,...,747.364348,601.999820,7.273432,1.041300,28.309999,109.879997,5954.500000,18847.279297,4.202,16.968000
2025-03,2986.409528,21656.9,319.799,181005700,7959.297917,3287.155300,2399.287715,2384.706841,2250.641087,2187.522538,...,744.113867,599.381567,7.249252,1.081314,30.990000,107.559998,5611.850098,17299.289062,4.210,21.841429
2025-04,3211.152367,21804.7,320.795,354296400,8211.983788,3391.513462,2475.458517,2460.414741,2322.092766,2256.970377,...,767.737440,618.410287,7.296782,1.123177,29.600000,104.370003,5569.060059,17446.339844,4.175,31.966190
